In [ ]:
# imports
import json
import jsonlines
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
import seaborn as sb
from statistics import mean

In [ ]:
BORDA_BASE = "../new_data_gen/Data/history_file_synthetic_baseline_borda.json"
BORDA_LOT = "../new_data_gen/Data/history_file_synthetic_product_lottery_borda.json"
BORDA_WEIGHT = "../new_data_gen/Data/history_file_synthetic_weighted_alloc_borda.json"
BORDA_LEAST = "../new_data_gen/Data/history_file_synthetic_least_fair_borda.json"

COPELAND_BASE = "../new_data_gen/Data/history_file_synthetic_baseline_copeland.json"
COPELAND_LOT = "../new_data_gen/Data/history_file_synthetic_product_lottery_copeland.json"
COPELAND_WEIGHT = "../new_data_gen/Data/history_file_synthetic_weighted_alloc_copeland.json"
COPELAND_LEAST = "../new_data_gen/Data/history_file_synthetic_least_fair_copeland.json"

RP_BASE = "../new_data_gen/Data/history_file_synthetic_baseline_rankedpairs.json"
RP_LOT = "../new_data_gen/Data/history_file_synthetic_product_lottery_rankedpairs.json"
RP_WEIGHT = "../new_data_gen/Data/history_file_synthetic_weighted_alloc_rankedpairs.json"
RP_LEAST = "../new_data_gen/Data/history_file_synthetic_least_fair_rankedpairs.json"

WR_BASE = "../new_data_gen/Data/history_file_synthetic_baseline_weighted_rescore.json"
WR_LEAST = "../new_data_gen/Data/history_file_synthetic_least_fair_weighted_rescore.json"
WR_LOT = "../new_data_gen/Data/history_file_synthetic_product_lottery_weighted_rescore.json"
WR_WEIGHT = "../new_data_gen/Data/history_file_synthetic_weighted_alloc_weighted_rescore.json"

recs_file = "../new_data_gen/Data/recs.csv"
items_file = "../new_data_gen/Data/items.csv"

In [ ]:
# reads recommender and item files
recommender = pd.read_csv(recs_file, names=["User","Item","Score"])
item_features = pd.read_csv(items_file, names=["Item","Feature","BV"])

In [ ]:
# no need to change unless adding a new allocation mechanism
history_files = [BORDA_BASE, BORDA_LOT, BORDA_WEIGHT, BORDA_LEAST, COPELAND_BASE, COPELAND_LEAST,COPELAND_LOT,COPELAND_WEIGHT,RP_BASE, RP_LEAST,RP_LOT,RP_WEIGHT, WR_BASE, WR_LEAST, WR_LOT, WR_WEIGHT]

borda_b =[]
borda_lo = []
borda_w= []
borda_le = []
copeland_b =[]
copeland_le= []
copeland_lo = []
copeland_w= []
rp_b = []
rp_le = []
rp_lo= []
rp_w= []
wr_b = []
wr_le = []
wr_lo = []
wr_w = []



list_names = [borda_b, borda_lo, borda_w,borda_le,copeland_b, copeland_le,copeland_lo,copeland_w,rp_b,rp_le,rp_lo,rp_w,wr_b,wr_le,wr_lo,wr_w]
readable_names = ['Borda Baseline','Borda Lottery', 'Borda Weighted Product', 'Borda Least Fair', 'Copeland Baseline','Copeland Least Fair', 'Copeland Lottery', 'Copeland Weighted Product','Ranked Pairs Baseline','Ranked Pairs Least Fair', 'Ranked Pairs Lottery', 'Ranked Pairs Weighted Product', 'Weighted Rescore Baseline', 'Weighted Rescore Lottery', 'Weighted Rescore Product', 'Weighted Rescore Least Fair']

In [ ]:
# define dictionaries for statistical analysis, change if adding new allocation mechanisms

results_borda_b = {}
results_borda_lo = {}
results_borda_w= {}
results_borda_le = {}
results_copeland_b = {}
results_copeland_le= {}
results_copeland_lo = {}
results_copeland_w= {}
results_rp_b = {}
results_rp_le = {}
results_rp_lo = {}
results_rp_w = {}
results_wr_b = {}
results_wr_le = {}
results_wr_lo = {}
results_wr_w = {}


dict_names = [results_borda_b,results_borda_lo, results_borda_w, results_borda_le, results_copeland_b,results_copeland_le, results_copeland_lo, results_copeland_w, results_rp_b,results_rp_le, results_rp_lo, results_rp_w, results_wr_b, results_wr_le, results_wr_lo, results_wr_w]

In [ ]:
# read history files
for history_file, list_name in zip(history_files,list_names):
    with jsonlines.open(history_file) as reader:
        for obj in reader:
            list_name.append(obj)

In [ ]:
# process history files into dictionaries for statistics
for list, dictionary in zip(list_names, dict_names):
    for line in list:
        results = line['choice_out']['results']
        results_list = []
        for item in results:
            results_list.append(item['item'])
        dictionary[line['user']] = results_list 

In [ ]:
### STATISTICS FUNCTIONS ###

# get recommendation score based on user and item ids
def lookupscore(user, item):
    user = int(user)
    item = int(item)
    score = recommender.loc[(recommender.User == user) & (recommender.Item == item)]["Score"]
    return float(score)

# calculate ndcg given a list of recommended and ideal scores
def ndcg(scores1, scores2):
    idealdcg = 0.0
    recdcg = 0.0
    for index, val in enumerate(scores1):
        recdcg += (2**val - 1)/np.log2(index + 2)
    for index, val in enumerate(scores2):
         idealdcg += (2**val - 1)/np.log2(index + 2)
    return recdcg/idealdcg

# not currently run in this file
def plot_ndcg(name,ndcg_results):
    ndcg_result = ndcg_results[name]
    ndcg_data = pd.DataFrame(ndcg_result)
    sb.lineplot(ndcg_data)
    imagefile = name + "ndcgplot.png"
    plt.savefig(imagefile)

# given an item id return a list of its features as binary values
def get_item_features(item_id):
    feature_values = []
    for value in item_features.loc[(item_features.Item == int(item_id))]["BV"]:
        feature_values.append(value)
    return feature_values



In [ ]:
### VISUALIZATION FUNCTIONS ###
# TODO: Update viz functions to make them more efficient and inline with statistics

def process_history(history, fair=True, compat=True, alloc=True, lists=True):
    if fair:
        fair_list = [entry['allocation']['fairness scores'] for entry in history]
        fair_df = pd.DataFrame(fair_list)
    else:
        fair_df = None
    if compat:
        compat_list = [entry['allocation']['compatibility scores'] for entry in history]
        compat_df = pd.DataFrame(compat_list)
    else:
        compat_df = None
    if alloc:
        alloc_list = [entry['allocation']['output'] for entry in history]
        alloc_df = pd.DataFrame(alloc_list)
        alloc_df['none'] = (alloc_df['1'] == 0) & (alloc_df['2'] == 0)
    else:
        alloc_df = None
    if lists:
        results_list = [process_results(entry['choice_out']['results']) for entry in history]
    else:
        results_list = None
    return fair_df, compat_df, alloc_df, results_list

def process_results(result_structs):
    return [(entry['item'], entry['score']) for entry in result_structs]

def plot_fairness_time(experiment_data, include_none=False, image_prefix=None):

    fair_df = experiment_data[0]
    fig, ax = plt.subplots(figsize=(10, 6))
    sb.set(font_scale=2)
    plt.xlabel("Time")
    plt.ylabel("Fairness")
    sb.lineplot(data=fair_df)
    image_file = image_prefix + '-fairness.png'
    plt.savefig(image_file)

def plot_allocation(experiment_data, include_none=False, image_prefix=None):
    alloc_df = pd.DataFrame(experiment_data[2])
    if include_none is False:
        if not alloc_df['none'][1:].any():
            alloc_df.drop('none', axis=1, inplace=True)
    fig, ax = plt.subplots(figsize=(10, 6))
    sb.set(font_scale=2)
    plt.xlabel("Time")
    plt.ylabel("Allocation")
    sb.lineplot(data=alloc_df.cumsum())
    image_file = image_prefix + '-allocation.png'
    plt.savefig(image_file)

def plot_fairness_regret(experiment_data, include_none=False, image_prefix=None):

    fair_df = experiment_data[0]
    regret = 1-fair_df
    fig, ax = plt.subplots(figsize=(10, 6))
    sb.set(font_scale=2)
    plt.xlabel("Time")
    plt.ylabel("Fairness Regret")
    sb.lineplot(data=regret.cumsum())
    image_file = image_prefix + '-regret.png'
    plt.savefig(image_file)

def do_plots(experiment_data, include_none=False, image_prefix=None):
    plot_fairness_time(experiment_data, include_none, image_prefix)
    plot_allocation(experiment_data, include_none, image_prefix)
    plot_fairness_regret(experiment_data, include_none, image_prefix)

def process(experiment, include_none=False, image_prefix=None):
    experiment_data = process_history(experiment)
    do_plots(experiment_data, include_none, image_prefix)


In [ ]:
# generates average NDCG for each allocation mechanism and stores them in the list avg_of_ndcg
# also creates ndcg_results, a dictionary that allows plotting NDCG over time for each mechanism
avg_of_ndcg = []
ndcg_results = {}
for dictionary, name in zip(dict_names, readable_names):
    users = []
    ndcg_values = []
    for user, items in dictionary.items():
        scores = []
        for item in items:
            scores.append(lookupscore(user, item))
        ideal_scores = []
        for score in recommender.loc[(recommender.User == int(user))]["Score"].sort_values(ascending=False):
            ideal_scores.append(score)
            ideal_scores = ideal_scores[0:len(scores)+1]
        ndcg_values.append(ndcg(scores, ideal_scores))
        users.append(user)
    avg_of_ndcg.append(mean(ndcg_values))
    ndcg_results[name] = {"Users":users, "NDCG":ndcg_values}

In [ ]:
# creates dataframe of average NDCG values
ndcg_table = pd.DataFrame(data=avg_of_ndcg, index=readable_names, columns=["NDCG"])
ndcg_table

In [ ]:
# calculates proportional fairness for the representation of item features
# currently designed to handle two three features
# TODO: make it easier to change number of item features
either_fairness = []
f1_representation = []
f2_representation = []

for dictionary, name in zip(dict_names, readable_names):
    item_counter = 0
    f0 = 0
    f1 = 0
    f2 = 0
    either = 0
    for items in dictionary.values():
        for item in items:
            item_counter += 1
            list_of_features = get_item_features(item)
            if list_of_features[0] == 1:
                f0 += 1
            if list_of_features[1] == 1:
                f1 += 1
            if list_of_features[2] == 1:
                f2 += 1
            if list_of_features[1] == 1 or list_of_features[2] == 1:
                either += 1
    either_fairness.append(either/item_counter)
    f1_representation.append(f1/item_counter)
    f2_representation.append(f2/item_counter)

In [ ]:
# calculates average fairness in relation to threshold
agent1 = ['1']
prop1 = .25
agent2 = ['2']
prop2 = .25
fairness_results = {"Protected":either_fairness,"1":f1_representation,"2":f2_representation}
prop_fairness_results = pd.DataFrame(data=fairness_results, index=readable_names)
prop_fairness_results['score1'] = prop_fairness_results[agent1]/prop1
prop_fairness_results['score2'] = prop_fairness_results[agent2]/prop2
prop_fairness_results['Average Fairness Score'] = (prop_fairness_results['score1']+prop_fairness_results['score2'])/2

In [ ]:
ndcg_fairness = prop_fairness_results.merge(ndcg_table, left_index = True, right_index = True)

In [ ]:
ndcg_fairness

In [ ]:
baseline_accuracy = ndcg_fairness.iloc[0,5]

In [ ]:
ndcg_fairness.loc[0:4, "Choice"] = "Borda"
ndcg_fairness.loc[4:8, "Choice"] = "Copeland"
ndcg_fairness.loc[8:12, "Choice"] = "Ranked Pairs"
ndcg_fairness.loc[12:16, "Choice"] = "Rescoring"

In [ ]:
#ndcg_fairness["Allocation"] = ndcg_fairness.apply(lambda x: x.name.replace(x["Choice"], ''), axis=1)

In [ ]:
ndcg_fairness = ndcg_fairness.drop(["Borda Baseline", "Copeland Baseline", "Ranked Pairs Baseline", "Weighted Rescore Baseline"])

In [ ]:
ndcg_fairness["Allocation"] = ''

In [ ]:
ndcg_fairness.iloc[0,8] = "Lottery"
ndcg_fairness.iloc[1,8] = "Weighted"
ndcg_fairness.iloc[2,8] = "Least Fair"
ndcg_fairness.iloc[3:10:3,8] = "Least Fair"
ndcg_fairness.iloc[4:11:3,8] = "Lottery"
ndcg_fairness.iloc[5:12:3,8] = "Weighted"

In [ ]:
ndcg_fairness

In [ ]:
ndcg_fairness["Proportional Difference"] = ndcg_fairness.apply(lambda x: abs(x["1"] - x["2"]), axis=1)

In [ ]:
ndcg_fairness["Normalized Difference"] = ndcg_fairness.apply(lambda x: abs(x["score1"] - x["score2"]), axis=1)

In [ ]:
difference_table = ndcg_fairness[["Proportional Difference", "Normalized Difference"]]

In [ ]:
difference_table = difference_table[0:16]

In [ ]:
sb.set_style("white")
plot = sb.scatterplot(x='Average Fairness Score',
                      y='NDCG',
                      data=ndcg_fairness,
                      style="Allocation",
                      hue="Choice",
                      s=80)

sb.set(font_scale = 1.0)
#plt.axhline(y = baseline_accuracy, linestyle="dashed")
plt.xlabel("Average Fairness Score", fontsize=11, labelpad=10)
plt.ylabel("nDCG", fontsize=10, labelpad=11)
plt.legend(loc="lower left", fontsize=10.5)